In [11]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from spacy.lang.en import English
parser = English()
from bs4 import BeautifulSoup as bs
import nltk
import string
from nltk.corpus import stopwords
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_lg')
from collections import Counter

In [2]:
# Load data
train1 = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [9]:
train1.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


In [11]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train1, test_size=0.33, random_state=42)
print('Training Data Shape:', train.shape)
print('Testing Data Shape:', val.shape)

Training Data Shape: (1732, 3)
Testing Data Shape: (854, 3)


In [5]:
STOPLIST = set(list(ENGLISH_STOP_WORDS))
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”"]
"""
def clean(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = text.lower()
    return text
"""
train1['description'] = train1['description'].apply(lambda x: bs(x).get_text().replace("\\n", " " ).strip("b'").lower())

def tokenize(sample):
    tokens = parser(sample)
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    return tokens

In [6]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train1, test_size=0.33, random_state=42)
print('Training Data Shape:', train.shape)
print('Testing Data Shape:', val.shape)

Training Data Shape: (1732, 3)
Testing Data Shape: (854, 3)


In [7]:
# Create pipeline components
vect = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,2), min_df=5, max_df=0.5)
clf = LinearSVC()

In [8]:
# Define the Pipeline
pipe = Pipeline([
                 ('vect', vect), 
                 ('clf', clf)
                ])

In [9]:
x_train_fit = train1['description']
y_train_fit = train1['category']
x_train = train['description']
y_train = train['category']
x_val = val['description']
y_val = val['category']
x_test = test['description']

In [18]:
# Define the Pipeline
pipe = Pipeline([
                 ('vect', vect), 
                 ('clf', clf)
                ])

pipe.fit(x_train_fit, y_train_fit)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [15]:
prediction = pipe.predict(x_val)
print("accuracy:", accuracy_score(y_val, prediction))

accuracy: 0.9437939110070258


In [19]:
test_pred = pipe.predict(x_test)


In [20]:
submission = pd.DataFrame({'id': test['id'], 'category': test_pred})
submission.to_csv('submission1.csv', index=0)

In [ ]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])